In [1]:
import pandas as pd
from geopy.distance import distance
import networkx as nx

In [2]:
STORAGE_SIZE_SHARE = 0.05
LATENCY_PER_KM = 0.0048
NETWORK_FACTOR = 1.7
DATA_PROCESSING_MS = 0

Функция для расчета задержки между двумя узлами

In [3]:
# point -> (lat, lon)
def get_latency(point1, point2):
    return round(distance(point1, point2).km * LATENCY_PER_KM * NETWORK_FACTOR + DATA_PROCESSING_MS, 2)

Загружаем граф, задаем веса ребрам и сохраняем кратчайшие пути между узлами

In [4]:
G = nx.read_gml("topology.gml")

# Сервер, который содержит все видео, а также все узлы сети обращаются к нему в самом начале
ORIGIN_SERVER = "DLLS"

# Задаем веса всем ребрам графа - задержки между узлами сети
for A_label, B_label in G.edges:
    A = G.nodes[A_label]
    B = G.nodes[B_label]
    G[A_label][B_label]['weight'] = get_latency((A['Latitude'], A['Longitude']), (B['Latitude'], B['Longitude']))
    print(f"{A_label} <- {G[A_label][B_label]['weight']} -> {B_label}")


# Определяем кратчайшие пути между всеми узлами и длину этих путей
fast_path = {}
fast_path_length = {}
for src, (lengths, paths) in nx.all_pairs_dijkstra(G):
    fast_path[src] = {}
    for dst, path in paths.items():
        fast_path[src][dst] = path

    fast_path_length[src] = {}
    for dst, length in lengths.items():
        fast_path_length[src][dst] = length

nodes = list(G.nodes)
nodes.remove(ORIGIN_SERVER)

NY54 <- 2.48 -> CMBR
NY54 <- 9.37 -> CHCG
NY54 <- 1.06 -> PHLA
NY54 <- 2.68 -> WASH
CMBR <- 3.54 -> PHLA
CHCG <- 4.06 -> CLEV
CHCG <- 8.75 -> PHLA
CHCG <- 3.41 -> STLS
CHCG <- 12.07 -> DNVR
CHCG <- 5.42 -> KSCY
CHCG <- 24.4 -> SNFN
CHCG <- 22.81 -> STTL
CHCG <- 16.54 -> SLKC
CLEV <- 6.03 -> NSVL
CLEV <- 6.46 -> STLS
CLEV <- 4.71 -> PHLA
RLGH <- 4.67 -> ATLN
RLGH <- 3.06 -> WASH
ATLN <- 7.12 -> WASH
ATLN <- 2.82 -> NSVL
ATLN <- 6.14 -> STLS
ATLN <- 9.48 -> DLLS
ATLN <- 5.26 -> ORLD
PHLA <- 1.63 -> WASH
NSVL <- 3.32 -> STLS
NSVL <- 8.11 -> DLLS
STLS <- 7.19 -> DLLS
STLS <- 3.17 -> KSCY
STLS <- 20.87 -> LA03
NWOR <- 4.17 -> HSTN
NWOR <- 5.82 -> DLLS
NWOR <- 7.01 -> ORLD
HSTN <- 2.49 -> SNAN
HSTN <- 2.95 -> DLLS
HSTN <- 11.15 -> ORLD
SNAN <- 11.14 -> PHNX
SNAN <- 3.31 -> DLLS
DLLS <- 8.69 -> DNVR
DLLS <- 5.95 -> KSCY
DLLS <- 19.48 -> SNFN
DLLS <- 16.28 -> LA03
DNVR <- 7.29 -> KSCY
DNVR <- 12.47 -> SNFN
DNVR <- 4.88 -> SLKC
KSCY <- 19.76 -> SNFN
SNFN <- 0.99 -> SCRM
SNFN <- 7.02 -> PTLD
SNF

Загружаем датасет с городами и определяем средневзвешенную по численности населения задержку между абстрактным клиентом и узлом

In [5]:
cities = pd.read_csv("uscities.csv")

# Для каждого города определяем ближайший узел сети
node_cities = {i: [] for i in nodes}
i = 1
for city in cities.iloc:
    point1 = (city["lat"], city["lng"])
    population = city["population"]
    min_latency = -1
    min_node = ""
    for target_node in nodes:
        point2 = (G.nodes[target_node]['Latitude'], G.nodes[target_node]['Longitude'])
        latency = get_latency(point1, point2)
        if min_latency == -1 or latency < min_latency:
            min_latency = latency
            min_node = target_node
    node_cities[min_node].append((min_latency, population))
    print(f"\r{i}/{cities.shape[0]}", end='')
    i += 1

# Считаем средневзвешенную по численности населения задержку от абстрактного клиента до узла
node_city_latency = {}
for target_node, values in node_cities.items():
    pop_lat = 0
    sum_pop = 0
    for lat, pop in values:
        pop_lat += pop * lat
        sum_pop += pop
    node_city_latency[target_node] = round(pop_lat / sum_pop, 2)

node_city_latency

28338/28338

{'NY54': 0.32,
 'CMBR': 0.55,
 'CHCG': 1.72,
 'CLEV': 1.55,
 'RLGH': 1.45,
 'ATLN': 1.11,
 'PHLA': 0.42,
 'WASH': 0.62,
 'NSVL': 1.59,
 'STLS': 1.11,
 'NWOR': 1.35,
 'HSTN': 1.92,
 'SNAN': 1.77,
 'ORLD': 2.41,
 'DNVR': 1.33,
 'KSCY': 2.41,
 'SNFN': 4.2,
 'SCRM': 0.83,
 'PTLD': 0.79,
 'STTL': 1.94,
 'SLKC': 1.44,
 'LA03': 0.68,
 'SNDG': 0.45,
 'PHNX': 1.32}

Загружаем датасет с видео, определяем популярность видео в виде доли просмотров от общего числа, определяем ограничение по размеру хранилища узла

In [6]:
videos = pd.read_csv("./videos.csv")
# Чтобы не работало долго берем небольшую выборку
videos = videos.sample(200)
videos["popularity"] = videos["view_count"] / videos["view_count"].sum()

# Размер хранилища узла определяется как доля от суммарного размера видео в датасете
NODE_STORAGE_SIZE = int(videos["video_size"].sum() * STORAGE_SIZE_SHARE)

Подготавливаем данные, которые алгоритм будет изменять в процессе работы

In [7]:
# Текущее свободное место в каждом узле
node_capacity = {node: NODE_STORAGE_SIZE for node in nodes}
# Текущая задержка на каждом узле: задержка от абстрактного клиента до узла + средневзвешенная по популярности задержка для получения видео
# Изначально заполняется только задержка от абстрактного клиента до узла
node_latency = {node: node_city_latency[node] for node in nodes}
# Ближайший к текущему узлу узел содержащий копию видео
node_video_closest_server = {node: {} for node in nodes}
# Список видео содержащихся на узле
node_videos = {node: [] for node in nodes}

# Прибавляем средневзвешенную по популярности задержку для получения видео
i = 0
for target_node in nodes:
    j = 1
    for video in videos.iloc:
        node_latency[target_node] += fast_path_length[target_node][ORIGIN_SERVER] * video["popularity"]
        node_video_closest_server[target_node][video["video_id"]] = ORIGIN_SERVER
        print(f"\r{j + i * videos.shape[0]}/{videos.shape[0] * len(nodes)}", end='')
        j += 1
    i += 1

node_latency

4800/4800

{'NY54': 19.59999999999999,
 'CMBR': 22.31000000000001,
 'CHCG': 12.32,
 'CLEV': 15.2,
 'RLGH': 15.599999999999996,
 'ATLN': 10.589999999999998,
 'PHLA': 18.650000000000013,
 'WASH': 17.220000000000002,
 'NSVL': 9.700000000000003,
 'STLS': 8.300000000000004,
 'NWOR': 7.170000000000003,
 'HSTN': 4.869999999999999,
 'SNAN': 5.079999999999998,
 'ORLD': 15.239999999999993,
 'DNVR': 10.019999999999996,
 'KSCY': 8.360000000000001,
 'SNFN': 23.68,
 'SCRM': 21.29999999999999,
 'PTLD': 27.290000000000003,
 'STTL': 30.32999999999999,
 'SLKC': 15.010000000000002,
 'LA03': 16.96,
 'SNDG': 18.190000000000012,
 'PHNX': 15.770000000000003}

Алгоритм распределения видео по узлам сети

In [8]:
# Копия текущего словаря с задержками. Используется в алгоритме
node_with_space_latency = node_latency.copy()

while node_with_space_latency:
    # Выбираем узел с наибольшей задержкой
    target_node = max(node_with_space_latency, key=node_with_space_latency.get)

    # Ищем видео, получение которого происходит с наибольшей задержкой и размер которого не превышает текущее свободное место на узле
    max_video_latency = 0
    target_video = None
    for video in videos.iloc:
        video_node = node_video_closest_server[target_node][video["video_id"]]
        if video_node == target_node:
            continue
        cur_latency = fast_path_length[target_node][video_node] * video["popularity"]
        if cur_latency > max_video_latency and node_capacity[target_node] > video["video_size"]:
            max_video_latency = cur_latency
            target_video = video
    # Означает, что остались только видео, которые не влезают на текущий узел
    if target_video is None:
        node_with_space_latency.pop(target_node)
        continue
    for node in nodes:
        # Определяем будет ли улучшение для узла от размещения видео на выбранном узле
        cur_video_node = node_video_closest_server[node][target_video["video_id"]]
        cur_latency = fast_path_length[node][cur_video_node]
        new_latency = fast_path_length[node][target_node]
        if cur_latency < new_latency:
            continue

        # Обновляем задержку и меняем узел, с которого данное видео будет запрашиваться
        node_latency[node] -= cur_latency * target_video["popularity"]
        node_latency[node] += new_latency * target_video["popularity"]
        node_video_closest_server[node][target_video["video_id"]] = target_node
        if node in node_with_space_latency:
            node_with_space_latency[node] -= cur_latency * target_video["popularity"]
            node_with_space_latency[node] += new_latency * target_video["popularity"]
    # Добавляем видео на узел и уменьшаем свободное место
    node_videos[target_node].append(target_video["video_id"])
    node_capacity[target_node] -= target_video["video_size"]

node_latency

{'NY54': 4.899295284315782,
 'CMBR': 5.378884131942755,
 'CHCG': 5.193903675892968,
 'CLEV': 5.101776549420974,
 'RLGH': 5.126231426493415,
 'ATLN': 4.482210762222275,
 'PHLA': 4.571531474463566,
 'WASH': 4.574670986366336,
 'NSVL': 4.35525672791589,
 'STLS': 3.706427680923964,
 'NWOR': 3.566073315830426,
 'HSTN': 3.129522702649116,
 'SNAN': 3.0679940742094542,
 'ORLD': 6.872549671867417,
 'DNVR': 5.073508578007596,
 'KSCY': 4.53854391451263,
 'SNFN': 8.231768352566807,
 'SCRM': 5.483167127378758,
 'PTLD': 7.283203412597586,
 'STTL': 8.099164368391756,
 'SLKC': 5.603487400324568,
 'LA03': 4.526610682961157,
 'SNDG': 4.608837091737104,
 'PHNX': 5.416730684154542}